# Smart Parser

Ziel ist ein Parser, der einfache Sätze besser versteht. Eingaben sind gewöhnliche Text-Adventure-Sätze, Ausgaben sind die Commands.

Der Parser benötigt mehrere Stufen um die Syntax zu verstehen, diese mit den erlaubten Verben zu Matchen und die Objekte zu identifizieren.

Verwendet werden:
- SpaCy: Model zur Analyse und Annotation der eingegebenen Sätze. Vermutlich "de_dep_news_trf".
- SentenceTransformer: Zum Matching der Commands, vermutlich mit "paraphrase-multilingual-MiniLM-L12-v2".
- Neo4J: Zum identifizieren der Objekte.

In [27]:
import os
import spacy 
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer, util

load_dotenv(dotenv_path='../.env')

parsing_model_trf = spacy.load("de_dep_news_trf")
matching_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

driver = GraphDatabase.driver(
    uri=os.getenv('NEO4J_URI'),
    auth=(
        os.getenv('NEO4J_USER'), 
        os.getenv('NEO4J_PASSWORD')
    )
)

print('Docker starten ;-)')
print(f'URI: {os.getenv("NEO4J_URI")}')
print(f'UI:  http://localhost:7474')

Docker starten ;-)
URI: bolt://localhost:7687
UI:  http://localhost:7474


In [28]:
# model response ansehen
doc_trf = parsing_model_trf("Nimm den goldenen Schlüssel und öffne die verzauberte Truhe")

spacy.displacy.render(doc_trf, style='dep', jupyter=True)

## Testsätze

KI-generierte Sätze verschiedener Kategorien inklusive expected outcome.

In [29]:
basic = [
    {
        'sentence': "Nimm den Kristall",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    },
    {
        'sentence': "Lege die Laterne ab",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Geh zur Höhle",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Untersuche den Hammer",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [[]]}
    },
    {
        'sentence': "Lies die Gravur",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}
    },
    {
        'sentence': "Benutze den Dolch",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    }
]

trennbar = [
    {
        'sentence': "Nimm das Edelstein auf",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    },
    {
        'sentence': "Wirf die Lampe weg",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Lauf zur Grotte",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Sieh dir das Werkzeug an",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [[]]}
    },
    {
        'sentence': "Les die Inschrift vor",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}
    },
    {
        'sentence': "Wende das Messer an",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    }
]

komplex = [
    {
        'sentence': "Nimm den leuchtenden Kristall",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [['leuchtenden']]}
    },
    {
        'sentence': "Lege die flackernde alte Laterne ab",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [['flackernde', 'alte']]}
    },
    {
        'sentence': "Geh in die dunkle Höhle",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [['dunkle']]}
    },
    {
        'sentence': "Untersuche den rostigen Hammer",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [['rostigen']]}
    },
    {
        'sentence': "Lies die alte Gravur",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [['alte']]}
    },
    {
        'sentence': "Öffne die Kiste mit dem scharfen Dolch",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [['scharfen']]}
    }
]

praepositionen = [
    {
        'sentence': "Hole den Edelstein aus der Spalte",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    },
    {
        'sentence': "Lege die Lampe auf den Boden",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Gehe in die Grotte",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Sieh dir das Werkzeug an der Wand an",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [[]]}
    },
    {
        'sentence': "Lies die Inschrift auf dem Stein",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}
    },
    {
        'sentence': "Öffne die Truhe mit dem Messer",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    }
]

synonyme = [
    {
        'sentence': "Greif nach dem glänzenden Stein",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [['glänzenden']]}
    },
    {
        'sentence': "Lass die Leuchte fallen",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Besuche die Kaverne",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Betrachte den Schmiedehammer",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [[]]}
    },
    {
        'sentence': "Durchlese die Runen",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}
    },
    {
        'sentence': "Verwende die Klinge",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    }
]

schwierig = [
    {
        'sentence': "Schnapp dir den funkelnden Brocken",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [['funkelnden']]}
    },
    {
        'sentence': "Schmeiß das Licht weg",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Mach dass du zur Felsenhöhle kommst",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Guck dir das schwere Ding mal genauer an",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [['schwere']]}
    },
    {
        'sentence': "Was steht da drauf?",
        'expected': {'command': 'read', 'objects': [], 'adjectives': []}
    },
    {
        'sentence': "Probier mal die Waffe aus",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    }
]

# Neue Kategorie: Fuzzy Synonyme (stark abweichende Bezeichnungen)
fuzzy_synonyme = [
    {
        'sentence': "Nimm das Juwel",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    },
    {
        'sentence': "Nimm das Mineral",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    },
    {
        'sentence': "Leg die Fackel ab",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Schmeiß die Lichtquelle weg",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Geh ins Loch",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Besuch die Felsenkammer",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Schau dir das metallische Ding an",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [['metallische']]}
    },
    {
        'sentence': "Untersuche das schwere Teil",
        'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [['schwere']]}
    },
    {
        'sentence': "Lies was da steht",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}
    },
    {
        'sentence': "Entziffere die Schrift",
        'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}
    },
    {
        'sentence': "Benutze die Hiebwaffe",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    },
    {
        'sentence': "Nimm den Degen",
        'expected': {'command': 'take', 'objects': ['dolch'], 'adjectives': [[]]}
    },
    {
        'sentence': "Verwende den Säbel",
        'expected': {'command': 'use', 'objects': ['dolch'], 'adjectives': [[]]}
    },
    {
        'sentence': "Pack das scharfe Teil ein",
        'expected': {'command': 'take', 'objects': ['dolch'], 'adjectives': [['scharfe']]}
    }
]

edge_cases = [
    {
        'sentence': "Nimm Kristall",
        'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    },
    {
        'sentence': "Laterne ablegen",
        'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    },
    {
        'sentence': "Ich möchte zur Höhle gehen",
        'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
    },
    {
        'sentence': "Kannst du dir das ansehen?",
        'expected': {'command': 'examine', 'objects': [], 'adjectives': []}
    },
    {
        'sentence': "",
        'expected': None,
        'setup': None
    }
]

all_tests = {
    'basic': basic,
    'trennbar': trennbar,
    'komplex': komplex,
    'praepositionen': praepositionen,
    'synonyme': synonyme,
    'fuzzy_synonyme': fuzzy_synonyme,
    'schwierig': schwierig,
    'edge_cases': edge_cases
}

## NLP Processing

In [ ]:
# Parser für einzelne Items aus den Testdaten
def add_trf_parsing(sentence):

    results = {}

    doc = parsing_model_trf(sentence)
    
    # Fügt Verb und Objekte den Testdaten hinzu
    results['nlp_verb'] = None
    results['nlp_objects'] = []
    results['nlp_adj'] = []
    
    for token in doc:
        # Hauptverb finden (lemma_)
        if token.dep_ == "ROOT" and token.dep_ == "VERB":
            results['nlp_verb'] = token.lemma_
            
        # Objekte finden
        if token.dep_ in ['obj', 'dobj', 'oa', 'pobj']:
            results['nlp_objects'] = token.text

            # Adjektiv finden wenn vorhanden
            for child in token.children:
                if child.pos_ == 'ADJ':
                    results['nlp_adj'] = child.text

    return results

In [31]:
for category, tests in all_tests.items():

    for items in tests:

        items['nlp_results'] = add_trf_parsing(items['sentence'])

In [32]:
print(all_tests)

{'basic': [{'sentence': 'Nimm den Kristall', 'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': 'Kristall', 'nlp_adj': []}}, {'sentence': 'Lege die Laterne ab', 'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': 'Laterne', 'nlp_adj': []}}, {'sentence': 'Geh zur Höhle', 'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': [], 'nlp_adj': []}}, {'sentence': 'Untersuche den Hammer', 'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': 'Hammer', 'nlp_adj': []}}, {'sentence': 'Lies die Gravur', 'expected': {'command': 'read', 'objects': ['gravur'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': [], 'nlp_adj': []}}, {'sentence': 'Benutze den Dolch', 'expected': {'command': 'use',

# Command Embedding

In [33]:
# Command-Struktur:
# Meta (hardcoded): inventory, quit, help, look
# In-World (geparst): Commands unten

command_verbs = {
    # Beobachtung
    'examine': [
        'untersuchen', 'betrachten', 'ansehen', 'anschauen', 'inspizieren', 'prüfen', 'mustern',
        'untersuch', 'betracht', 'sieh an', 'schau an', 'guck an'  # Imperativ
    ],
    'read': [
        'lesen', 'durchlesen', 'vorlesen',
        'lies', 'les'  # Imperativ
    ],
    
    # Bewegung
    'go': [
        'gehen', 'laufen', 'bewegen', 'besuchen', 'kommen',
        'geh', 'lauf', 'beweg', 'besuch', 'komm'  # Imperativ
    ],
    
    # Objekt-Interaktion
    'take': [
        'nehmen', 'holen', 'packen', 'greifen', 'schnappen', 'aufheben', 'raffen',
        'nimm', 'hol', 'pack', 'greif', 'schnapp'  # Imperativ
    ],
    'drop': [
        'ablegen', 'werfen', 'lassen', 'fallenlassen', 'wegwerfen', 'schmeißen',
        'leg ab', 'wirf', 'lass', 'schmeiß'  # Imperativ
    ],
    'use': [
        'benutzen', 'verwenden', 'anwenden', 'öffnen', 'betätigen', 'probieren', 'aktivieren',
        'benutz', 'verwend', 'wend an', 'öffne', 'probier'  # Imperativ
    ],
}

# Commands embedden
command_verb_embeddings = {}

for cmd, verbs in command_verbs.items():
    command_verb_embeddings[cmd] = matching_model.encode(verbs)

In [ ]:
# Embedding und Vergleich der Verben mit den Commands
def verb_to_command(verb):

    result = {}

    # Abbruch wenn kein Verb gefunden
    if verb is None:
        result['best_command'] = None        
        result['best_sim'] = 0.0
        return

    result['best_command'] = None
    result['best_sim'] = -1

    # Verb embedden
    verb_emb = matching_model.encode(verb)   

    for cmd, embs in command_verb_embeddings.items():

        # Ähnlichkeit vergleichen
        similarities = util.cos_sim(verb_emb, embs)
        max_sim = similarities.max().item()

        # Bestes Ergebnis schreiben wenn > -1
        if max_sim > result['best_sim']:
            result['best_sim'] = max_sim
            result['best_command'] = cmd
    
    return result

# Smart Parsing

"Einmal mit alles"

In [35]:
for category, tests in all_tests.items():

    for items in tests:

        items['parsed_command'] = verb_to_command(items['sentence'])

In [36]:
print(all_tests)

{'basic': [{'sentence': 'Nimm den Kristall', 'expected': {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': 'Kristall', 'nlp_adj': []}, 'parsed_command': {'best_command': 'take', 'best_sim': 0.5612605214118958}}, {'sentence': 'Lege die Laterne ab', 'expected': {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': 'Laterne', 'nlp_adj': []}, 'parsed_command': {'best_command': 'drop', 'best_sim': 0.844207763671875}}, {'sentence': 'Geh zur Höhle', 'expected': {'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': [], 'nlp_adj': []}, 'parsed_command': {'best_command': 'go', 'best_sim': 0.5912449955940247}}, {'sentence': 'Untersuche den Hammer', 'expected': {'command': 'examine', 'objects': ['hammer'], 'adjectives': [[]]}, 'nlp_results': {'nlp_verb': None, 'nlp_objects': 'Hammer', 'nlp_adj': []}, 'parsed_command': 

## AI Analyse

Danke Claude :)

In [37]:
# Fehleranalyse: Expected vs. Predicted
from collections import defaultdict

errors = defaultdict(lambda: defaultdict(int))

for category, tests in all_tests.items():
    for item in tests:
        if item.get('expected') is None:
            continue
        
        expected = item['expected']['command']
        predicted = item.get('parsed_command', {}).get('best_command')
        
        # Zähle alle Kombinationen (auch korrekte)
        errors[expected][predicted] += 1

# Nur Fehler anzeigen
print("\n" + "="*80)
print("FEHLERANALYSE: Expected → Predicted (nur Fehler)")
print("="*80)

total_errors = 0
for expected in sorted(errors.keys()):
    has_errors = False
    error_list = []
    
    for predicted, count in sorted(errors[expected].items(), key=lambda x: (x[0] is None, x[0])):
        if expected != predicted:  # Nur Fehler
            pred_str = str(predicted) if predicted else "None"
            error_list.append(f"{pred_str}({count}x)")
            total_errors += count
            has_errors = True
    
    if has_errors:
        correct = errors[expected].get(expected, 0)
        total = sum(errors[expected].values())
        print(f"\n{expected.upper():12} ({correct}/{total} korrekt)")
        print(f"  Verwechselt mit: {', '.join(error_list)}")

print(f"\n{'='*80}")
print(f"Gesamt-Fehler: {total_errors}")

# Confusion Matrix (optional, detaillierter)
print("\n" + "="*80)
print("CONFUSION MATRIX")
print("="*80)

# Nur nicht-None Commands
all_commands = sorted(set(errors.keys()) | {pred for preds in errors.values() for pred in preds.keys() if pred is not None})

# Header
print(f"{'Expected':12}", end='')
for cmd in all_commands:
    print(f"{cmd:>10}", end='')
print(f"{'None':>10}")
print("-"*80)

# Rows
for expected in all_commands:
    print(f"{expected:12}", end='')
    for predicted in all_commands:
        count = errors[expected].get(predicted, 0)
        if expected == predicted and count > 0:
            print(f"{count:>10}", end='')  # Korrekte
        elif count > 0:
            print(f"\033[91m{count:>10}\033[0m", end='')  # Fehler in rot
        else:
            print(f"{'':>10}", end='')
    
    # None-Spalte
    none_count = errors[expected].get(None, 0)
    if none_count > 0:
        print(f"\033[91m{none_count:>10}\033[0m", end='')
    else:
        print(f"{'':>10}", end='')
    print()

print("\n" + "="*80)


FEHLERANALYSE: Expected → Predicted (nur Fehler)

DROP         (7/9 korrekt)
  Verwechselt mit: examine(2x)

EXAMINE      (7/9 korrekt)
  Verwechselt mit: take(2x)

READ         (7/8 korrekt)
  Verwechselt mit: go(1x)

TAKE         (7/11 korrekt)
  Verwechselt mit: drop(2x), examine(1x), use(1x)

USE          (4/8 korrekt)
  Verwechselt mit: drop(1x), take(3x)

Gesamt-Fehler: 13

CONFUSION MATRIX
Expected          drop   examine        go      read      take       use      None
--------------------------------------------------------------------------------
drop                 7         2                                                  
examine                        7                             2                    
go                                       9                                        
read                                     1         7                              
take                 2         1                             7         1          
use                 

In [38]:
# Treffer ansehen
print("\n" + "="*80)
print(" "*25 + "SMART PARSER TEST REPORT")
print("="*80)

# Per Command Accuracy
from collections import defaultdict

command_stats = defaultdict(lambda: {'total': 0, 'correct': 0})

for category, tests in all_tests.items():
    for item in tests:
        if item.get('expected') is None:
            continue

        expected_cmd = item['expected']['command']
        predicted_cmd = item.get('parsed_command', {}).get('best_command')

        command_stats[expected_cmd]['total'] += 1
        if expected_cmd == predicted_cmd:
            command_stats[expected_cmd]['correct'] += 1

print("\nACCURACY PRO COMMAND:")
print('-'*80)
for cmd, stats in sorted(command_stats.items()):
    acc = 100 * stats['correct'] / stats['total'] if stats['total'] > 0 else 0
    bar = '█' * int(acc / 5)  # Visual bar
    print(f"{cmd:<10} {stats['correct']:>2}/{stats['total']:<2} ({acc:>5.1f}%) {bar}")

print("\n" + "="*80)
print(" "*25 + "ACCURACY PRO SENTENCE")
print("="*80)

for category, tests in all_tests.items():

    print(f"\nKATEGORIE: {category.upper()}")
    print('-'*80)

    for i, item in enumerate(tests, 1):
        print(f"\n[{i}] {item['sentence']}")
        
        if item.get('expected'):
            print(f"    Expected:  {item['expected']}")

        # NLP Results
        if item.get('nlp_results'):
            nlp = item['nlp_results']
            verb = nlp.get('nlp_verb', 'N/A')
            
            # Objekte formatieren
            objects = nlp.get('nlp_objects')
            if isinstance(objects, list):
                obj_str = [obj.get('text', str(obj)) if isinstance(obj, dict) else str(obj) for obj in objects]
            else:
                obj_str = [str(objects)] if objects else []
            
            print(f"    Parsed:    verb='{verb}', objects={obj_str}")

        # Command Matching
        if item.get('parsed_command'):
            cmd = item['parsed_command']
            predicted_cmd = cmd.get('best_command')
            score = cmd.get('best_sim', 0)
            
            if item.get('expected'):
                expected_cmd = item['expected']['command']
                match_icon = '✓' if expected_cmd == predicted_cmd else '✗'
                print(f"    Predicted: {predicted_cmd} (score: {score:.3f}) {match_icon}")
            else:
                print(f"    Predicted: {predicted_cmd} (score: {score:.3f})")
        else:
            print(f"    Predicted: N/A")



                         SMART PARSER TEST REPORT

ACCURACY PRO COMMAND:
--------------------------------------------------------------------------------
drop        7/9  ( 77.8%) ███████████████
examine     7/9  ( 77.8%) ███████████████
go          9/9  (100.0%) ████████████████████
read        7/8  ( 87.5%) █████████████████
take        7/11 ( 63.6%) ████████████
use         4/8  ( 50.0%) ██████████

                         ACCURACY PRO SENTENCE

KATEGORIE: BASIC
--------------------------------------------------------------------------------

[1] Nimm den Kristall
    Expected:  {'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
    Parsed:    verb='None', objects=['Kristall']
    Predicted: take (score: 0.561) ✓

[2] Lege die Laterne ab
    Expected:  {'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
    Parsed:    verb='None', objects=['Laterne']
    Predicted: drop (score: 0.844) ✓

[3] Geh zur Höhle
    Expected:  {'command': 'go', 'objects': ['hoeh

## Objekt Matching

Bezeichnung, Beschreibung und Synonyme werden/wurden embedded und sollen genutzt werden.

In [39]:
# Queryhelper
def run_query(query, params=None):
    
    # Ergebnisliste
    results = []

    # Session öffnen
    with driver.session() as session:

        # Query
        result = session.run(query, params or {})

        # Ergebnisse wegspeichern
        records = [r.data() for r in result]
        results.extend(records)

    return results

# Objektsuche
def find_object_in_db(object):

    # Auf leer prüfen
    if not object or object == []:
        return [], [], []

    emb_object = matching_model.encode(object)
    params = {'object_embedding': emb_object.tolist()}

    name_query = """
        MATCH (p:Player {id: 'testplayer'})-[:IST_IN]->(loc:Location)
        CALL db.index.vector.queryNodes('item_name_index', 5, $object_embedding)
        YIELD node AS item, score
        WHERE (item)-[:IST_IN]->(loc)
        RETURN item.id, item.name, score
        LIMIT 1
        """
    
    name_result = run_query(name_query, params)
    
    description_query = """
        MATCH (p:Player {id: 'testplayer'})-[:IST_IN]->(loc:Location)
        CALL db.index.vector.queryNodes('item_description_index', 5, $object_embedding)
        YIELD node AS item, score
        WHERE (item)-[:IST_IN]->(loc)
        RETURN item.id, item.name, score
        LIMIT 1
        """

    description_result = run_query(description_query, params)

    synonyme_query = """
        MATCH (p:Player {id: 'testplayer'})-[:IST_IN]->(loc:Location)
        CALL db.index.vector.queryNodes('item_synonyms_index', 5, $object_embedding)
        YIELD node AS item, score
        WHERE (item)-[:IST_IN]->(loc)
        RETURN item.id, item.name, score
        LIMIT 1
        """

    synonyme_result = run_query(synonyme_query, params)

    return name_result, description_result, synonyme_result

In [40]:
for category, tests in all_tests.items():
    for items in tests:
        items['retrived_names'], items['retrived_description'], items['retrived_synonyms'] = find_object_in_db(items['nlp_results']['nlp_objects'])

In [41]:
for category, tests in all_tests.items():
    for items in tests:
        print(items['sentence'])
        print(items['expected'])
        print(items['nlp_results'])
        print(items['parsed_command'])
        print(items['retrived_names'])
        print(items['retrived_description'])
        print(items['retrived_synonyms'])
        print()

Nimm den Kristall
{'command': 'take', 'objects': ['kristall'], 'adjectives': [[]]}
{'nlp_verb': None, 'nlp_objects': 'Kristall', 'nlp_adj': []}
{'best_command': 'take', 'best_sim': 0.5612605214118958}
[{'item.id': 'test_kristall', 'item.name': 'Kristall', 'score': 1.0}]
[{'item.id': 'test_kristall', 'item.name': 'Kristall', 'score': 0.7891967296600342}]
[{'item.id': 'test_schluessel', 'item.name': 'Goldener Schlüssel', 'score': 0.7597260475158691}]

Lege die Laterne ab
{'command': 'drop', 'objects': ['laterne'], 'adjectives': [[]]}
{'nlp_verb': None, 'nlp_objects': 'Laterne', 'nlp_adj': []}
{'best_command': 'drop', 'best_sim': 0.844207763671875}
[{'item.id': 'test_laterne', 'item.name': 'Laterne', 'score': 0.9999999403953552}]
[{'item.id': 'test_dolch', 'item.name': 'Dolch', 'score': 0.9029256105422974}]
[{'item.id': 'test_schluessel', 'item.name': 'Goldener Schlüssel', 'score': 0.7602344751358032}]

Geh zur Höhle
{'command': 'go', 'objects': ['hoehle'], 'adjectives': [[]]}
{'nlp_verb'